In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
pwd

'/Users/shashank/Documents/Python_Stuff/End-to-end-project-implementation'

In [4]:
#  We don't need any configuration in evaluation

In [7]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.h5")

2023-12-09 12:58:13.519750: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-12-09 12:58:13.519813: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-12-09 12:58:13.519823: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-12-09 12:58:13.519881: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-09 12:58:13.519899: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [13]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
                 self.config = read_yaml(config_filepath)
                 self.params = read_yaml(params_filepath)
                 create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
                    path_of_model= "artifacts/training/model.h5",
                    training_data= "artifacts/data_ingestion/Chicken-fecal-images/",
                    all_params= self.params,
                    params_image_size= self.params.IMAGE_SIZE,
                    params_batch_size= self.params.BATCH_SIZE
            )

        return eval_config

In [14]:
from urllib.parse import urlparse

In [15]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)


In [16]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2023-12-09 13:58:04,105: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-09 13:58:04,108: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-09 13:58:04,109: INFO: common: created directory at: artifacts]
Found 116 images belonging to 2 classes.


2023-12-09 13:58:04.923228: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - 2s 217ms/step - loss: 19.6352 - accuracy: 0.5000
[2023-12-09 13:58:07,123: INFO: common: json file saved at: scores.json]
